# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## CREATE TABLE FOR QUERY 1
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1_create = "CREATE TABLE IF NOT EXISTS song_details "
query1_create = query1_create + "(sessionId int, itemInSession int, artist varchar, song varchar, length float, "
query1_create = query1_create + "PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1_create)
except Exception as e:
    print(e)

In [9]:
## INSERT DATA FOR QUERY 1
file = 'event_datafile_new.csv'
query1_insert = "INSERT INTO song_details (sessionId, itemInSession, artist, song, length)"
query1_insert = query1_insert + " VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query1_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into song_details

In [10]:
## SELECT DATA FOR QUERY 1
query1_select = "SELECT artist, song, length FROM song_details WHERE sessionId = 338 AND itemInSession = 4"
try:
    query1_result = session.execute(query1_select);
except Exception as e:
    print(e)
    
for row in query1_result:
    print(row.artist, row.song, row.length,)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
## CREATE TABLE FOR QUERY 2

## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2_create = "CREATE TABLE IF NOT EXISTS artist_details "
query2_create = query2_create + "(userId int, sessionId int, itemInSession int, artist varchar, song varchar, firstName varchar, lastName varchar, "
query2_create = query2_create + "PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query2_create)
except Exception as e:
    print(e)

In [12]:
## INSERT DATA FOR QUERY 2
file = 'event_datafile_new.csv'
query2_insert = "INSERT INTO artist_details (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
query2_insert = query2_insert + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query2_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into artist_details

In [13]:
## SELECT DATA FOR QUERY 2
query2_select = "SELECT artist, song, firstName, lastName FROM artist_details WHERE userId = 10 AND sessionId = 182"
try:
    query2_result = session.execute(query2_select);
except Exception as e:
    print(e)
    
for row in query2_result:
    print(row.artist, row.song, row.firstname, row.lastname,)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## CREATE TABLE FOR QUERY 3
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3_create = "CREATE TABLE IF NOT EXISTS user_details "
query3_create = query3_create + "(song varchar, userId int, firstName varchar, lastName varchar, "
query3_create = query3_create + "PRIMARY KEY ((song), userId))"
try:
    session.execute(query3_create)
except Exception as e:
    print(e)                 

In [15]:
## INSERT DATA FOR QUERY 3
file = 'event_datafile_new.csv'
query3_insert = "INSERT INTO user_details (song, userId, firstName, lastName)"
query3_insert = query3_insert + " VALUES (%s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query3_insert, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into user_details

In [16]:
## SELECT DATA FOR QUERY 3
query3_select = "SELECT firstName, lastName FROM user_details WHERE song = 'All Hands Against His Own'"
try:
    query3_result = session.execute(query3_select);
except Exception as e:
    print(e)
    
for row in query3_result:
    print(row.firstname, row.lastname,)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## DROP TABLES
query1_drop = "DROP TABLE IF EXISTS song_details"
query2_drop = "DROP TABLE IF EXISTS artist_details"
query3_drop = "DROP TABLE IF EXISTS user_details"
try:
    session.execute(query1_drop)
    session.execute(query2_drop)
    session.execute(query3_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()